In [2]:
import pandas as pd
import pandas_datareader.data as pdr
import datetime
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np


print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

1.0.1.post2
Populating the interactive namespace from numpy and matplotlib


In [ ]:

 
# We will look at stock prices over the past year, starting at January 1, 2016
start = (2000, 12, 1)
start = datetime.datetime(*start)
end = datetime.date.today()

google = pdr.DataReader('028050.KS', 'yahoo', start, end)

In [ ]:
google.Low.plot(grid=True)

In [ ]:
google.tail()
i = 50
a = google.iloc[100:200]
a = a.apply(lambda x: (x-x[i]) /  x[i])
a

In [26]:
class StockDataset(Dataset):
    
    def __init__(self, symbol, x_frames, y_frames, start, end):
        
        self.symbol = symbol
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = datetime.datetime(*start)
        self.end = datetime.datetime(*end)

        self.data = pdr.DataReader(self.symbol, 'yahoo', self.start, self.end)
        
    def __len__(self):
        return len(self.data) - (self.x_frames + self.y_frames) + 1
    
    def __getitem__(self, idx):
        idx += self.x_frames
        data = self.data.iloc[idx-self.x_frames:idx+self.y_frames]
        data = data[['High', 'Low', 'Open', 'Close', 'Adj Close', 'Volume']]
        data = data.apply(lambda x: x / x[self.x_frames] - 1)
        data = data.values
        X = data[:self.x_frames]
        y = data[self.x_frames:]
        
        return {'X':X, 'y': y}
        


In [27]:
dataset = StockDataset('028050.KS', 5, 5, (2000,2,1), (2019, 2,1))
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

In [28]:
for batch in dataloader:
    X = batch['X']
    y = batch['y']
    print(X.shape, y.shape)
    print(X[0])
    break

torch.Size([10, 5, 6]) torch.Size([10, 5, 6])
tensor([[-0.1219, -0.0687, -0.0266, -0.1423, -0.1423, -0.4655],
        [-0.1219, -0.0969, -0.0609, -0.1239, -0.1239, -0.4342],
        [-0.1399, -0.1250, -0.0297, -0.2113, -0.2113, -0.5379],
        [-0.1828, -0.1656, -0.1656, -0.1930, -0.1930, -0.6284],
        [-0.0942, -0.1500, -0.1109, -0.1127, -0.1127, -0.1015]],
       dtype=torch.float64)
